In [159]:
import pandas as pd
from sqlite3 import connect
import plotly.express as px

# Get WHO data

In [160]:
who_url = r'https://covid.ourworldindata.org/data/full_data.csv'

In [161]:
who_data = pd.read_csv(who_url)
print(who_data.head(3))
with connect("covid-19.db") as conn:
  who_data.to_sql('who_data',conn,if_exists="replace")

         date     location  new_cases  new_deaths  total_cases  total_deaths
0  2020-02-25  Afghanistan        NaN         NaN            1           NaN
1  2020-02-26  Afghanistan        0.0         NaN            1           NaN
2  2020-02-27  Afghanistan        0.0         NaN            1           NaN


# Plot Data

In [162]:
px.line(who_data,x='date',y='total_cases',color='location').write_html('raw_plot.html',auto_open=True)

In [170]:
def plot_who(starting_threshold=100):
    sql = '''select who_data.location, who_data.date, who_data.total_cases, who_data.new_cases,
      julianday(who_data.date) - julianday(day_n.date) as day 
      from who_data left join (select location, min(date) as date from who_data where total_cases > ? GROUP by location) as day_n
      on who_data.location = day_n.location 
      where day is not NULL and day >= 0 and who_data.location != "International"'''
    with connect("covid-19.db") as conn:
        px.line(pd.read_sql(sql,conn,params=[starting_threshold]),
                x='day',y='total_cases',
                hover_name='location',
                hover_data=['total_cases','new_cases','date'],
                color='location',
                range_x = [0,10],
                range_y = [0,7500]
               ).write_html(f"plot-who-{starting_threshold}.html",auto_open=True)

In [171]:
plot_who(50)

# Get John Hopkins data

In [172]:
john_hopkins_url = r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'

In [173]:
jh_data = pd.read_csv(john_hopkins_url)
jh_data.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,43,47,48,50,50,50,53,59,70,75
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,331,360,420,461,502,511,581,639,639,701
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,110,117,130,138,150,150,160,178,178,200


In [174]:
with connect("covid-19.db") as conn:
  pd.melt(
      jh_data, 
      id_vars=['Country/Region','Province/State','Lat','Long'],
      var_name='date',
      value_name="confirmed"
      ).astype( {'date': 'datetime64[ns]'}
      ).rename( columns={"Country/Region": 'country','Province/State':'province'}
      ).to_sql( 'jh_data', conn,if_exists="replace")
  

In [181]:
def plot_jh(starting_threshold=100):
    sql = '''select jh_agg.country, jh_agg.date, jh_agg.confirmed,
      julianday(jh_agg.date) - julianday(day_n.date) as day 
      from (select country, date, sum(confirmed) as confirmed from jh_data group by country, date) as jh_agg
      left join 
      (select country, min(date) as date from jh_data where confirmed > ? GROUP by country) as day_n
      on jh_agg.country = day_n.country 
      where day is not null and day >= 0'''
    with connect("covid-19.db") as conn:
        px.line(pd.read_sql(sql,conn,params=[starting_threshold]),
                x = 'day',y = 'confirmed',
                hover_name = 'country',
                color = 'country',
                hover_data = ['confirmed','date'],
                range_x = [0, 10],
                range_y = [0, 7500]
               ).write_html(f"plot-jh-{starting_threshold}.html",auto_open=True)

In [182]:
plot_jh(50)